In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import pandas_datareader.data as web
import time
import tensorflow
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import logging
import math
import os
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras import optimizers
from sklearn.metrics import mean_squared_error
from keras.models import load_model

Using TensorFlow backend.


In [2]:
np.random.seed(8)

# 자료 불러오기

In [3]:
finance_info = pd.read_excel('./data/10년치_재무_LSTM_비율.xlsx')
stock_info = pd.read_excel('./data/10년치_주가_LSTM_비율.xlsx')

# 재무정보 파악

In [4]:
finance_info = finance_info.loc[:,:'*희석주당순손익(요약)(3개월)(IFRS연결)(원)']

In [5]:
finance_info['거래년도'] = finance_info['회계년도'].str.split('/',1).str[0]
finance_info['거래월'] = finance_info['회계년도'].str.split('/',1).str[1]

In [6]:
finance_info.columns

Index(['회사명', '거래소코드', '회계년도', '자산(*)(요약)(IFRS연결)(백만원)',
       '자본(*)(요약)(IFRS연결)(백만원)', '자본금(요약)(IFRS연결)(백만원)',
       '신종자본증권(요약)(IFRS연결)(백만원)', '자본잉여금(요약)(IFRS연결)(백만원)',
       '기타자본(요약)(IFRS연결)(백만원)', '기타포괄손익누계액(요약)(IFRS연결)(백만원)',
       '이익잉여금(결손금)(요약)(IFRS연결)(백만원)', '부채(*)(요약)(IFRS연결)(백만원)',
       '판매비와 관리비(물류원가 등 포함)(요약)(3개월)(IFRS연결)(백만원)',
       '* (정상)영업손익(보고서기재)(요약)(3개월)(IFRS연결)(백만원)',
       '* (정상)영업손익(계산수치)(요약)(3개월)(IFRS연결)(백만원)', '기타이익(요약)(3개월)(IFRS연결)(백만원)',
       '기타손실(요약)(3개월)(IFRS연결)(백만원)', '법인세비용차감전순이익(손실)(요약)(3개월)(IFRS연결)(백만원)',
       '법인세비용(요약)(3개월)(IFRS연결)(백만원)', '종속회사 매수일전 순손익(요약)(3개월)(IFRS연결)(백만원)',
       '처분된 종속회사 순손익(요약)(3개월)(IFRS연결)(백만원)',
       '계속영업이익(손실)(요약)(3개월)(IFRS연결)(백만원)', '중단영업이익(손실)(요약)(3개월)(IFRS연결)(백만원)',
       '*중단영업손익 법인세효과(요약)(3개월)(IFRS연결)(백만원)',
       '당기순이익(손실)(요약)(3개월)(IFRS연결)(백만원)', '기타포괄손익(요약)(3개월)(IFRS연결)(백만원)',
       '총포괄손익(요약)(3개월)(IFRS연결)(백만원)', '*주당계속영업이익(요약)(3개월)(IFRS연결)(원)',
       '*주당순손익(요약)(3개월)(IFRS연결)(원)', '*희석주당계속영업이익

In [7]:
finance_info.columns = ['회사명', '거래소코드', '회계년도', '자산(백만원)', '자본(백만원)', 
                             '자본금(백만원)', '신종자본증권(백만원)', '자본잉여금(백만원)', '기타자본(백만원)',
                             '기타포괄손익누계액(백만원)','이익잉여금(백만원)', '부채(백만원)', '판매비와 관리비(백만원)',
                             '영업손익(보고서기재)(백만원)', '영업손익(계산수치)(백만원)', '기타이익(백만원)','기타손실(백만원)',
                             '법인세비용차감전순이익(백만원)', '법인세비용(백만원)', '종속회사 매수일전 순손익(백만원)',
                             '처분된 종속회사 순손익(백만원)', '계속영업이익(손실)(백만원)','중단영업이익(손실)(백만원)',
                             '중단영업손익 법인세효과(백만원)', '당기순이익(손실)(백만원)', '기타포괄손익(백만원)','총포괄손익(백만원)',
                             '주당계속영업이익(원)','주당순손익(원)', '희석주당계속영업이익(원)','희석주당순손익(원)', '거래년도', '거래월']

In [8]:
finance_info = finance_info[['회사명', '거래소코드', '회계년도', '거래년도', '거래월', '자산(백만원)', '자본(백만원)', 
                             '자본금(백만원)', '신종자본증권(백만원)', '자본잉여금(백만원)', '기타자본(백만원)',
                             '기타포괄손익누계액(백만원)','이익잉여금(백만원)', '부채(백만원)', '판매비와 관리비(백만원)',
                             '영업손익(보고서기재)(백만원)', '영업손익(계산수치)(백만원)', '기타이익(백만원)','기타손실(백만원)',
                             '법인세비용차감전순이익(백만원)', '법인세비용(백만원)', '종속회사 매수일전 순손익(백만원)',
                             '처분된 종속회사 순손익(백만원)', '계속영업이익(손실)(백만원)','중단영업이익(손실)(백만원)',
                             '중단영업손익 법인세효과(백만원)', '당기순이익(손실)(백만원)', '기타포괄손익(백만원)','총포괄손익(백만원)',
                             '주당계속영업이익(원)','주당순손익(원)', '희석주당계속영업이익(원)','희석주당순손익(원)']]

In [9]:
finance_info['거래년도'] = pd.to_numeric(finance_info['거래년도']).astype(np.float64)
finance_info['거래월'] = pd.to_numeric(finance_info['거래월']).astype(np.float64)

In [10]:
finance_info.head(2)

,회사명,거래소코드,회계년도,거래년도,거래월,자산(백만원),자본(백만원),자본금(백만원),신종자본증권(백만원),자본잉여금(백만원),...,계속영업이익(손실)(백만원),중단영업이익(손실)(백만원),중단영업손익 법인세효과(백만원),당기순이익(손실)(백만원),기타포괄손익(백만원),총포괄손익(백만원),주당계속영업이익(원),주당순손익(원),희석주당계속영업이익(원),희석주당순손익(원)
0,(주)BNK금융지주,138930,2011/12,2011.0,12.0,39358673.0,2954947.0,966899.0,0.0,46405.0,...,79264.0,0.0,0.0,79264.0,7371.0,86635.0,407.0,407.0,0.0,0.0
1,(주)BNK금융지주,138930,2012/12,2012.0,12.0,42891404.0,3255216.0,966899.0,0.0,46405.0,...,52065.0,0.0,0.0,52065.0,-8326.0,43740.0,270.0,270.0,0.0,0.0


# 주가정보 파악

In [11]:
stock_info.columns = ['회사명', '거래소코드', '회계년도', '거래년도', '거래월', '거래일수', '최고가(원)', '최저가(원)',
       '산술평균(원)', '종가(원)', '거래량(주)', '거래대금(원)']

In [12]:
stock_info.head(2)

,회사명,거래소코드,회계년도,거래년도,거래월,거래일수,최고가(원),최저가(원),산술평균(원),종가(원),거래량(주),거래대금(원)
0,(주)BNK금융지주,138930,2011/03,2011.0,3.0,2.0,15950.0,14900.0,15800.0,15900.0,3846397.0,5.972707e+08
1,(주)BNK금융지주,138930,2011/06,2011.0,6.0,21.0,16000.0,13900.0,14881.0,15100.0,15349506.0,2.281128e+09


In [13]:
#del stock_info['산술평균(원)']
del stock_info['거래일수']


In [14]:
stock_info.head(10)

,회사명,거래소코드,회계년도,거래년도,거래월,최고가(원),최저가(원),산술평균(원),종가(원),거래량(주),거래대금(원)
0,(주)BNK금융지주,138930,2011/03,2011.0,3.0,15950.0,14900.0,15800.0,15900.0,3846397.0,5.972707e+08
1,(주)BNK금융지주,138930,2011/06,2011.0,6.0,16000.0,13900.0,14881.0,15100.0,15349506.0,2.281128e+09
2,(주)BNK금융지주,138930,2011/09,2011.0,9.0,14350.0,11850.0,13035.0,13150.0,19843801.0,2.583716e+09
3,(주)BNK금융지주,138930,2011/12,2011.0,12.0,11950.0,10300.0,11126.0,11050.0,16665990.0,1.857773e+09
4,(주)BNK금융지주,138930,2012/03,2012.0,3.0,14850.0,13050.0,14052.0,13300.0,14370897.0,2.005390e+09
5,(주)BNK금융지주,138930,2012/06,2012.0,6.0,13050.0,11300.0,12198.0,12650.0,10799735.0,1.314425e+09
6,(주)BNK금융지주,138930,2012/09,2012.0,9.0,12850.0,11250.0,12033.0,12000.0,13859498.0,1.666625e+09
7,(주)BNK금융지주,138930,2012/12,2012.0,12.0,13900.0,11900.0,12900.0,13200.0,9487437.0,1.226757e+09
8,(주)BNK금융지주,138930,2013/03,2013.0,3.0,16900.0,14750.0,15400.0,15300.0,13476686.0,2.074446e+09
9,(주)BNK금융지주,138930,2013/06,2013.0,6.0,16200.0,13150.0,14616.0,14450.0,13221673.0,1.916348e+09


In [15]:
# 열 순서 바꿀때
# stock_info = stock_info[['회사명',  '거래소코드', '거래년월','year', 'month', '최고가(원)', '최저가(원)', '산술평균(원)', '종가(원)',
#        '거래량(주)', '거래대금(원)']]

In [16]:
finance_info.head(2)

,회사명,거래소코드,회계년도,거래년도,거래월,자산(백만원),자본(백만원),자본금(백만원),신종자본증권(백만원),자본잉여금(백만원),...,계속영업이익(손실)(백만원),중단영업이익(손실)(백만원),중단영업손익 법인세효과(백만원),당기순이익(손실)(백만원),기타포괄손익(백만원),총포괄손익(백만원),주당계속영업이익(원),주당순손익(원),희석주당계속영업이익(원),희석주당순손익(원)
0,(주)BNK금융지주,138930,2011/12,2011.0,12.0,39358673.0,2954947.0,966899.0,0.0,46405.0,...,79264.0,0.0,0.0,79264.0,7371.0,86635.0,407.0,407.0,0.0,0.0
1,(주)BNK금융지주,138930,2012/12,2012.0,12.0,42891404.0,3255216.0,966899.0,0.0,46405.0,...,52065.0,0.0,0.0,52065.0,-8326.0,43740.0,270.0,270.0,0.0,0.0


# 재무정보, 주가정보 merge

In [18]:
lstm_data = pd.merge(finance_info, stock_info, on = ['회사명','거래소코드','회계년도','거래년도','거래월'],how='inner')

In [19]:
lstm_data.head(2)

,회사명,거래소코드,회계년도,거래년도,거래월,자산(백만원),자본(백만원),자본금(백만원),신종자본증권(백만원),자본잉여금(백만원),...,주당계속영업이익(원),주당순손익(원),희석주당계속영업이익(원),희석주당순손익(원),최고가(원),최저가(원),산술평균(원),종가(원),거래량(주),거래대금(원)
0,(주)BNK금융지주,138930,2011/12,2011.0,12.0,39358673.0,2954947.0,966899.0,0.0,46405.0,...,407.0,407.0,0.0,0.0,11950.0,10300.0,11126.0,11050.0,16665990.0,1.857773e+09
1,(주)BNK금융지주,138930,2012/12,2012.0,12.0,42891404.0,3255216.0,966899.0,0.0,46405.0,...,270.0,270.0,0.0,0.0,13900.0,11900.0,12900.0,13200.0,9487437.0,1.226757e+09


In [20]:
lstm_data.isna().sum()

회사명                      0
거래소코드                    0
회계년도                     0
거래년도                     0
거래월                      0
자산(백만원)               2476
자본(백만원)               2476
자본금(백만원)              2476
신종자본증권(백만원)           3044
자본잉여금(백만원)            2476
기타자본(백만원)             2476
기타포괄손익누계액(백만원)        2476
이익잉여금(백만원)            2476
부채(백만원)               2476
판매비와 관리비(백만원)         3034
영업손익(보고서기재)(백만원)      2939
영업손익(계산수치)(백만원)       3263
기타이익(백만원)             2939
기타손실(백만원)             2939
법인세비용차감전순이익(백만원)      2939
법인세비용(백만원)            2939
종속회사 매수일전 순손익(백만원)    2939
처분된 종속회사 순손익(백만원)     2939
계속영업이익(손실)(백만원)       2939
중단영업이익(손실)(백만원)       2939
중단영업손익 법인세효과(백만원)     2939
당기순이익(손실)(백만원)        2939
기타포괄손익(백만원)           2939
총포괄손익(백만원)            2939
주당계속영업이익(원)           2939
주당순손익(원)              2939
희석주당계속영업이익(원)         2939
희석주당순손익(원)            2939
최고가(원)                   0
최저가(원)                   0
산술평균(원)                  0
종가(원)                    0
거

# 필요없는 열 제거

In [21]:
pd.DataFrame(lstm_data.columns)

,0
0,회사명
1,거래소코드
2,회계년도
3,거래년도
4,거래월
5,자산(백만원)
6,자본(백만원)
7,자본금(백만원)
8,신종자본증권(백만원)
9,자본잉여금(백만원)


In [22]:
# 7 8 9 10 16 18 21 22 24 25 29 30 31 32 행 제거


lstm_data = lstm_data.drop('자본금(백만원)',1) # 7
lstm_data = lstm_data.drop('신종자본증권(백만원)',1) # 8
lstm_data = lstm_data.drop('자본잉여금(백만원)',1) # 9
lstm_data = lstm_data.drop('기타자본(백만원)',1) # 10
lstm_data = lstm_data.drop('영업손익(계산수치)(백만원)',1) # 16 십육
lstm_data = lstm_data.drop('기타손실(백만원)',1) # 18 십팔
lstm_data = lstm_data.drop('종속회사 매수일전 순손익(백만원)',1) # 21
lstm_data = lstm_data.drop('처분된 종속회사 순손익(백만원)',1) # 22
lstm_data = lstm_data.drop('중단영업이익(손실)(백만원)',1) # 24
lstm_data = lstm_data.drop('중단영업손익 법인세효과(백만원)',1) # 25
lstm_data = lstm_data.drop('주당계속영업이익(원)',1) # 29
lstm_data = lstm_data.drop('주당순손익(원)',1) # 30
lstm_data = lstm_data.drop('희석주당계속영업이익(원)',1) # 31
lstm_data = lstm_data.drop('희석주당순손익(원)',1) # 32

## 회사별로 리스트에 담음

In [23]:
list_company = list(set(lstm_data['회사명']))

In [24]:
list_company_stock = []
for i in range(len(list_company)):
    temp = lstm_data[lstm_data['회사명'] == list_company[i]]
    temp = temp.sort_values('회계년도', ascending = True)
    list_company_stock.append(temp)

In [25]:
len(list_company_stock)

197

In [26]:
list_company_stock[0] # 각 리스트의 길이는 대부분 44개임

,회사명,거래소코드,회계년도,거래년도,거래월,자산(백만원),자본(백만원),기타포괄손익누계액(백만원),이익잉여금(백만원),부채(백만원),...,계속영업이익(손실)(백만원),당기순이익(손실)(백만원),기타포괄손익(백만원),총포괄손익(백만원),최고가(원),최저가(원),산술평균(원),종가(원),거래량(주),거래대금(원)
6951,씨제이제일제당(주),97950,2007/09,2007.0,9.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,254000.0,221000.0,230000.0,230000.0,509746.0,1.206814e+09
1316,씨제이제일제당(주),97950,2007/12,2007.0,12.0,2944270.0,715757.0,13522.0,16714.0,2228513.0,...,NaN,NaN,NaN,NaN,298500.0,273000.0,286306.0,296000.0,824201.0,2.353246e+09
3204,씨제이제일제당(주),97950,2008/03,2008.0,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,268000.0,211500.0,238143.0,220000.0,1118111.0,2.635715e+09
5075,씨제이제일제당(주),97950,2008/06,2008.0,6.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,277000.0,253500.0,266275.0,273500.0,577896.0,1.533009e+09
6952,씨제이제일제당(주),97950,2008/09,2008.0,9.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,247500.0,187000.0,219238.0,192500.0,1634950.0,3.354661e+09
1317,씨제이제일제당(주),97950,2008/12,2008.0,12.0,4706795.0,1595515.0,837210.0,36753.0,3111280.0,...,NaN,NaN,NaN,NaN,197000.0,140000.0,171643.0,179000.0,1764595.0,3.003086e+09
3205,씨제이제일제당(주),97950,2009/03,2009.0,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,160000.0,115000.0,141545.0,144000.0,2148453.0,3.066541e+09
5076,씨제이제일제당(주),97950,2009/06,2009.0,6.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,173000.0,142500.0,157273.0,149500.0,2425849.0,3.865378e+09
6953,씨제이제일제당(주),97950,2009/09,2009.0,9.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,219000.0,165500.0,187682.0,207500.0,3014449.0,5.739471e+09
1318,씨제이제일제당(주),97950,2009/12,2009.0,12.0,4877276.0,1837602.0,822428.0,297054.0,3039674.0,...,NaN,NaN,NaN,NaN,229500.0,201500.0,215881.0,205500.0,1572352.0,3.394718e+09


# 주식 중에서 시계열 데이터가 부족한 자료 찾기
### 각 리스트의 길이는 대부분 44개임 => 총 128개 주식

In [27]:
list_company_stock_44 = []
for i in range(len(list_company_stock)):
    if len(list_company_stock[i]) == 44:
        list_company_stock_44.append(list_company_stock[i])


list_company_stock = list_company_stock_44 
len(list_company_stock) # 128개

128

# 128개 주식 중 na 값이 없는 주식은 총 65개

In [28]:
list_company_stock_not_na = []
for i in range(len(list_company_stock)):
    if len(list_company_stock[i].dropna(axis = 0)) == 28:
        list_company_stock_not_na.append(list_company_stock[i].dropna(axis = 0))


# list_company_stock = list_company_stock_44 
len(list_company_stock_not_na)

### 2011년 3월부터 2017년 12월까지 데이터 존재

65

# cf. 변화율말고 값으로 우선 LSTM 모델 돌려보고 그 다음에 변화율로 LSTM 모델 돌려보자

# 변화율로 바꾸자

In [29]:
list_company_stock_not_na[0].head(2)

,회사명,거래소코드,회계년도,거래년도,거래월,자산(백만원),자본(백만원),기타포괄손익누계액(백만원),이익잉여금(백만원),부채(백만원),...,계속영업이익(손실)(백만원),당기순이익(손실)(백만원),기타포괄손익(백만원),총포괄손익(백만원),최고가(원),최저가(원),산술평균(원),종가(원),거래량(주),거래대금(원)
3153,쌍용양회공업(주),3410,2011/03,2011.0,3.0,2977720.0,1303104.0,-12884.0,154491.0,1674616.0,...,-42285.0,-42285.0,2479.0,-39805.0,10250.0,6110.0,7501.0,8150.0,24347807.0,2.032447e+09
5024,쌍용양회공업(주),3410,2011/06,2011.0,6.0,2956241.0,1324094.0,-13087.0,163703.0,1632147.0,...,13307.0,13307.0,92.0,13398.0,7250.0,6420.0,6704.0,6840.0,2542140.0,1.734351e+08


## 비율로 바꾸는 함수: pct_change()

In [30]:
for j in range(len(list_company_stock_not_na)):
    for i in range(len(list_company_stock_not_na[j].columns[5:])):
        list_company_stock_not_na[j][list_company_stock_not_na[j].columns[5:][i]] = list_company_stock_not_na[j][list_company_stock_not_na[j].columns[5:][i]].pct_change()
    
    # 비율로 바뀌었기에 첫 행이 NA가 되어서 이를 제거함
    list_company_stock_not_na[j] = list_company_stock_not_na[j].drop(list_company_stock_not_na[j].index[0],0) 

In [31]:
list_company_stock_not_na[0]

,회사명,거래소코드,회계년도,거래년도,거래월,자산(백만원),자본(백만원),기타포괄손익누계액(백만원),이익잉여금(백만원),부채(백만원),...,계속영업이익(손실)(백만원),당기순이익(손실)(백만원),기타포괄손익(백만원),총포괄손익(백만원),최고가(원),최저가(원),산술평균(원),종가(원),거래량(주),거래대금(원)
5024,쌍용양회공업(주),3410,2011/06,2011.0,6.0,-0.007213,0.016108,0.015756,0.059628,-0.025360,...,-1.314698,-1.314698,-0.962888,-1.336591,-0.292683,0.050736,-0.106252,-0.160736,-0.895591,-0.914667
6900,쌍용양회공업(주),3410,2011/09,2011.0,9.0,0.017128,-0.126141,0.041492,-0.039559,0.133356,...,-1.737131,-1.737131,-9.141304,-1.788028,-0.231724,-0.330218,-0.253729,-0.342836,-0.435503,-0.589021
1265,쌍용양회공업(주),3410,2011/12,2011.0,12.0,0.043995,-0.029156,0.001834,-0.382091,0.089752,...,-1.673565,-1.673565,13.963952,-0.564122,-0.176840,-0.081395,-0.147711,-0.081201,-0.204249,-0.313484
3154,쌍용양회공업(주),3410,2012/03,2012.0,3.0,0.005305,-0.042427,0.013182,-0.457376,0.031905,...,-7.635992,-7.635992,-1.050946,8.403086,0.299891,0.207595,0.223734,0.222760,0.841983,1.260617
5025,쌍용양회공업(주),3410,2012/06,2012.0,6.0,0.012157,0.024559,0.177593,0.486712,0.005744,...,-1.624304,-1.624304,-5.269702,-1.576179,-0.152685,-0.224319,-0.159448,-0.112871,-0.048496,-0.214891
6901,쌍용양회공업(주),3410,2012/09,2012.0,9.0,-0.004679,0.021141,0.085379,0.312000,-0.018281,...,-0.069999,-0.069999,-0.520098,-0.025990,0.152475,0.316216,0.211810,0.261161,-0.461105,-0.337247
1266,쌍용양회공업(주),3410,2012/12,2012.0,12.0,-0.004949,0.023983,-0.998643,0.088381,-0.020802,...,0.203960,0.203960,-7.691453,0.584394,0.017182,0.013347,0.011665,0.003540,-0.117243,-0.098367
3155,쌍용양회공업(주),3410,2013/03,2013.0,3.0,-0.011705,-0.038466,-1.083333,-0.392652,0.003629,...,-2.406715,-2.406715,-0.932942,-2.106817,0.349662,0.436677,0.400037,0.388007,0.397746,0.932087
5026,쌍용양회공업(주),3410,2013/06,2013.0,6.0,-0.003333,0.028143,-86.000000,0.853984,-0.020613,...,-1.538886,-1.538886,-0.880000,-1.547021,0.077597,-0.055007,0.011690,-0.108005,1.045950,1.105920
6902,쌍용양회공업(주),3410,2013/09,2013.0,9.0,-0.005761,0.031588,-5.782353,0.275966,-0.027285,...,0.538458,0.538458,2.841270,0.544729,-0.272938,-0.183582,-0.214942,-0.149573,-0.494668,-0.609514


## 164개 주식을 가지고 LSTM 모형 만들것

In [ ]:
list_code_company_1 = []

for i in range(len(list_code_company)):
    company1 = list_code_company[i]
    company1 = company1.reset_index()
    del company1['index']
    
    stock_price_1year = list_code_company[i]['종가(원)'][4:]
    stock_price_1year = stock_price_1year.reset_index()
    del stock_price_1year['index']
    stock_price_1year.columns = ['1년뒤종가(원)']
    
    a = pd.concat([company1, stock_price_1year], axis = 1)
    a = a.loc[:15,:]
    list_code_company_1.append(a)

In [ ]:
list_code_company_1[1]

In [ ]:
test = list_code_company_1[0].loc[:,list_code_company_1[0].columns[4:]]
test.head()

# LSTM 모델 : 1개 종목

모델 개요
- 단순하게 2013년 1분기 데이터를 변수로 보고, 2014년 1분기 데이터를 예측하는 모델
- 13,14,15년은 Train
- 16은 Test  
=> 즉 16년 1분기를 보고 17년 1분기 예측

In [ ]:
'''
## import pandas as pd
import pandas_datareader.data as web
import tensorflow
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import logging
import math
import os
from sklearn.preprocessing import MinMaxScaler
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras import optimizers
from sklearn.metrics import mean_squared_error
from keras.models import load_model
from keras.utils import np_utils

### 데이터 전처리 / Data Preprocessing

np.random.seed(123)
# finance_indicator = test.loc[:,"자산(백만원)":].astype(np.float) # 재부 관련 지표
# month_indicator = test.loc[:,"month"].astype(np.float) # 월 관련 지표

indicator = test.values.astype(np.float)
scaler = MinMaxScaler(feature_range=(0, 1)) # 0~1 값으로 스케일링
scaled_test = scaler.fit_transform(indicator) # 가격 관련 지표에 스케일링
#month_indicator = np_utils.to_categorical(month_indicator)

#scaler_etc = MinMaxScaler(feature_range=(-1, 1)) # 0~1 값으로 스케일링
#scaled_etc_indicator = scaler_etc.fit_transform(etc_indicator) # 추세 또는 거래량 활용 지표에 스케일링



x = scaled_test
y = x[:, [25]] # 타켓은 1년뒤 종가이다 / target is 'Close' price


# dataX와 dataY 생성 / Creating dataset 'dataX' and 'dataY'
seq_length = 4 # 4분기 데이터를 이용해 1년뒤 예상

dataX = [] # 입력으로 사용될 Sequence Data / Input - Sequence Data
dataY = [] # 출력(타켓)으로 사용 / Output(target)
for i in range(0, int(len(y) - seq_length)):
    _x = x[i : i + seq_length]
    _y = y[i + seq_length - 1] # 다음 나타날 주가(정답) / after 5 days later (in _x), 'Close' price is the correct answer.
    dataX.append(_x) # dataX 리스트에 추가 / add to dataX's list
    dataY.append(_y) # dataY 리스트에 추가 / add to dataY's list
    print(_x[0:4])
    print('--------------')
    print(_y[0])
    print('--------------')
    
    

# 학습용/테스트용 데이터 생성
train_data_size = 0.67 # 입력하는 데이터 중에서 60퍼센트는 train 데이터로 사용
train_size = int(len(dataY) * train_data_size)
test_size = len(dataY) - train_size  # test 데이터: 4개

# 데이터를 잘라 학습용 데이터 생성
trainX = np.array(dataX[0:train_size])
trainY = np.array(dataY[0:train_size])

# 데이터를 잘라 테스트용 데이터 생성
testX = np.array(dataX[train_size:len(dataX)])
testY = np.array(dataY[train_size:len(dataY)])

print("="*50)
print(_x.shape)
print(_y.shape)
print(trainX.shape)
print(trainY.shape)
print("="*50)



## LSTM 모델

input_columns = 26

model = Sequential()
model.add(LSTM(64, input_dim=input_columns, input_length=seq_length, return_sequences=True, stateful=False))
model.add(LSTM(64, return_sequences=False, stateful=False))
model.add(Dense(1))
model.add(Activation('linear'))
# , dropout=0.2


# 모델 학습 설정 및 진행
keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()
print("="*50)
hist = model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=1, validation_data=(testX, testY))


# 학습 과정 살펴보기
print(hist.history['loss'])
print(hist.history['val_loss'])
fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()
loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')
plt.show()



# 예측
y_pred = model.predict(testX, batch_size=1, verbose=1) # , steps=5
plt.plot(testY, color = 'blue', label = 'Actual Stock Price')
plt.plot(y_pred, color = 'red', label = 'Predicted Stock Price')
plt.title('Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()


# 모델 저장
model.save('lstm_stock_prediction_01.h5')

# cf. 모델 로드하는거!
#model = load_model('lstm_stock_prediction_01.h5')
'''

# 모델2 : 기업 1개로만 한거

In [ ]:
# np.random.seed(125)


# test = list_code_company_1[0].loc[:,list_code_company_1[0].columns[4:]]
# test1 = test.loc[:,"month":"거래대금(원)"]
# ### 데이터 전처리 / Data Preprocessing
# # finance_indicator = test.loc[:,"자산(백만원)":].astype(np.float) # 재부 관련 지표
# # month_indicator = test.loc[:,"month"].astype(np.float) # 월 관련 지표

# # indicator = test.values.astype(np.float)
# indicator = test.values.astype(np.float)
# indicator1 = test1.values.astype(np.float)



# scaler = MinMaxScaler(feature_range=(0, 1)) # 0~1 값으로 스케일링

# scaled_test = scaler.fit_transform(indicator) # 가격 관련 지표에 스케일링
# scaled_test1 = scaler.fit_transform(indicator1)


# #month_indicator = np_utils.to_categorical(month_indicator)

# #scaler_etc = MinMaxScaler(feature_range=(-1, 1)) # 0~1 값으로 스케일링
# #scaled_etc_indicator = scaler_etc.fit_transform(etc_indicator) # 추세 또는 거래량 활용 지표에 스케일링


# x = scaled_test1
# y = scaled_test[:, [25]] # 타켓은 1년뒤 종가이다 / target is 'Close' price
# #print(len(x)) 16개
# #print(len(y))16개


# # dataX와 dataY 생성 / Creating dataset 'dataX' and 'dataY'
# seq_length = 4 # 4분기 데이터를 이용해 1년뒤 예상

# dataX = [] # 입력으로 사용될 Sequence Data / Input - Sequence Data
# dataY = [] # 출력(타켓)으로 사용 / Output(target)
# for i in range(0, int(len(y) - seq_length)):
#     _x = x[i : i + seq_length]
#     _y = y[i + seq_length - 1] # 다음해 나타날 주가(정답) / after 5 days later (in _x), 'Close' price is the correct answer.
#     dataX.append(_x) # dataX 리스트에 추가 / add to dataX's list
#     dataY.append(_y) # dataY 리스트에 추가 / add to dataY's list
#     print(_x)


# # 학습용/테스트용 데이터 생성
# train_data_size = 0.67 # 입력하는 데이터 중에서 60퍼센트는 train 데이터로 사용
# train_size = int(len(dataY) * train_data_size)
# test_size = len(dataY) - train_size  # test 데이터: 4개

# # 데이터를 잘라 학습용 데이터 생성
# trainX = np.array(dataX[0:train_size])
# trainY = np.array(dataY[0:train_size])

# # 데이터를 잘라 테스트용 데이터 생성
# testX = np.array(dataX[train_size:len(dataX)])
# testY = np.array(dataY[train_size:len(dataY)])

# print("="*50)
# print(_x.shape)
# print(_y.shape)
# print(trainX.shape)
# print(trainY.shape)
# print("="*50)

# # 모델 불러오기
# model = load_model('lstm_stock_prediction_01.h5')

# ## LSTM 모델

# input_columns = 25

# # 최초로는 모델 만들어야 하나 이후에는 학습시킬것
# model = Sequential()
# model.add(LSTM(64, input_dim=input_columns, input_length=seq_length, return_sequences=True, stateful=False))
# model.add(LSTM(64, return_sequences=False, stateful=False))
# model.add(Dense(1))
# model.add(Activation('linear'))
# # , dropout=0.2



# # 모델 학습 설정 및 진행
# keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# model.compile(loss='mean_squared_error', optimizer='adam')
# model.summary()
# print("="*50)
# hist = model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=1, validation_data=(testX, testY))


# # 학습 과정 살펴보기
# #     print(hist.history['loss'])
# #     print(hist.history['val_loss'])
# fig, loss_ax = plt.subplots()
# acc_ax = loss_ax.twinx()
# loss_ax.plot(hist.history['loss'], 'y', label='train loss')
# loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
# loss_ax.set_xlabel('epoch')
# loss_ax.set_ylabel('loss')
# loss_ax.legend(loc='upper left')
# acc_ax.legend(loc='lower left')
# plt.show()


# # 예측
# y_pred = model.predict(testX, batch_size=1, verbose=1) # , steps=5
# plt.plot(testY, color = 'blue', label = 'Actual Stock Price')
# plt.plot(y_pred, color = 'red', label = 'Predicted Stock Price')
# plt.title('Stock Price Prediction')
# plt.xlabel('Time')
# plt.ylabel('Stock Price')
# #plt.legend()
# plt.show()

# print(list_code_company_1[0]['회사명'].values[0])
# print('=====================================================')
# # 모델 저장
# model.save('lstm_stock_prediction_02_종가제외.h5')


# # cf. 모델 로드하는거!
# #model = load_model('lstm_stock_prediction_01.h5')

# 모델 2: for 문 - 모델 만들지 않고 로드해서 쓸것
  
### 기업 전부에 대해서 업데이트 시키는 모델

In [ ]:
np.random.seed(270)

for i in range(len(list_code_company_1)):

    test = list_code_company_1[i+24].loc[:,list_code_company_1[i+24].columns[4:]]
    test1 = test.loc[:,"month":"거래대금(원)"]
    ### 데이터 전처리 / Data Preprocessing
    # finance_indicator = test.loc[:,"자산(백만원)":].astype(np.float) # 재부 관련 지표
    # month_indicator = test.loc[:,"month"].astype(np.float) # 월 관련 지표

    # indicator = test.values.astype(np.float)
    indicator = test.values.astype(np.float) # 종가 뽑아내려고
    indicator1 = test1.values.astype(np.float) # 종가 제외한 피쳐



    scaler = MinMaxScaler(feature_range=(0, 1)) # 0~1 값으로 스케일링

    scaled_test = scaler.fit_transform(indicator) # 가격 관련 지표에 스케일링
    scaled_test1 = scaler.fit_transform(indicator1)


    #month_indicator = np_utils.to_categorical(month_indicator)

    #scaler_etc = MinMaxScaler(feature_range=(-1, 1)) # 0~1 값으로 스케일링
    #scaled_etc_indicator = scaler_etc.fit_transform(etc_indicator) # 추세 또는 거래량 활용 지표에 스케일링


    x = scaled_test1
    y = scaled_test[:, [25]] # 타켓은 1년뒤 종가이다 / target is 'Close' price
    #print(len(x)) 16개
    #print(len(y))16개


    # dataX와 dataY 생성 / Creating dataset 'dataX' and 'dataY'
    seq_length = 4 # 4분기 데이터를 이용해 1년뒤 예상

    dataX = [] # 입력으로 사용될 Sequence Data / Input - Sequence Data
    dataY = [] # 출력(타켓)으로 사용 / Output(target)
    for j in range(0, int(len(y) - seq_length)):
        _x = x[j : j + seq_length]
        _y = y[j + seq_length - 1] # 다음해 나타날 주가(정답) / after 5 days later (in _x), 'Close' price is the correct answer.
        dataX.append(_x) # dataX 리스트에 추가 / add to dataX's list
        dataY.append(_y) # dataY 리스트에 추가 / add to dataY's list
       


    # 학습용/테스트용 데이터 생성
    train_data_size = 0.67 # 입력하는 데이터 중에서 60퍼센트는 train 데이터로 사용
    train_size = int(len(dataY) * train_data_size)
    test_size = len(dataY) - train_size  # test 데이터: 4개

    # 데이터를 잘라 학습용 데이터 생성
    trainX = np.array(dataX[0:train_size])
    trainY = np.array(dataY[0:train_size])

    # 데이터를 잘라 테스트용 데이터 생성
    testX = np.array(dataX[train_size:len(dataX)])
    testY = np.array(dataY[train_size:len(dataY)])


    ## LSTM 모델

    input_columns = 25

    
    # 모델 로드
    model = load_model('./save/lstm_stock_prediction_02_종가제외.h5')


    # 모델 학습 설정 및 진행
    keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()
    print("="*50)
    hist = model.fit(trainX, trainY, epochs=50, batch_size=1, verbose=1, validation_data=(testX, testY))


    # 학습 과정 살펴보기
    #     print(hist.history['loss'])
    #     print(hist.history['val_loss'])
    fig, loss_ax = plt.subplots()
    acc_ax = loss_ax.twinx()
    loss_ax.plot(hist.history['loss'], 'y', label='train loss')
    loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc='upper left')
    acc_ax.legend(loc='lower left')
    plt.show()


    # 예측
    y_pred = model.predict(testX, batch_size=1, verbose=1) # , steps=5
    plt.plot(testY, color = 'blue', label = 'Actual Stock Price')
    plt.plot(y_pred, color = 'red', label = 'Predicted Stock Price')
    plt.title('Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    #plt.legend()
    plt.show()
    
    print(i+24,'번째')
    print(list_code_company_1[i+24]['회사명'].values[0])
    print('=====================================================')
    # 모델 저장
    model.save('./save/lstm_stock_prediction_02_종가제외.h5')
    time.sleep(0.5)

model.save('./save/lstm_stock_prediction_02_종가제외_완료.h5')

